## Extracción de datos de la API de idealista

#### Importación de librerías

In [ ]:
import pandas as pd
import os

import requests
from base64 import b64encode

#### Acceso a la API

In [2]:
'''Creo una función para obtener el access token según las instrucciones'''
def get_access_token(api_key, api_secret):
    
    credentials = f"{api_key}:{api_secret}"
    encoded_credentials = b64encode(credentials.encode()).decode('utf-8')

    token_url = "https://api.idealista.com/oauth/token"

    token_headers = {
        'Authorization': f'Basic {encoded_credentials}',
        'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

    token_data = {
        'grant_type': 'client_credentials',
        'scope': 'read'}

    token_response = requests.post(token_url, headers=token_headers, data=token_data)

    if token_response.status_code == 200:
        token_json = token_response.json()
        access_token = token_json['access_token']
        return access_token
    else:
        print(f"Error: {token_response.status_code}")
        print(token_response.text)
        return None
    

'''Creo la función donde hago la llamada segun los filtros de busqueda que desee aplicar'''
def search_properties(api_key, api_secret, country, operation, property_type, center, distance, locale='es', max_items=50, num_page=1, order='publicationDate', sort='desc', **filters):
    
    access_token = get_access_token(api_key, api_secret)
    
    if not access_token:
        return None

   
    search_url = f"https://api.idealista.com/3.5/{country}/search"

    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

    
    data = {
        'locale': locale,
        'maxItems': max_items,
        'numPage': num_page,
        'operation': operation,
        'order': order,
        'propertyType': property_type,
        'sort': sort,
        'center': center,
        'distance': distance,}

    # para añadir filtros adicionales
    data.update(filters)

    response = requests.post(search_url, headers=headers, data=data)

    if response.status_code == 200:
        json_response = response.json()
        return json_response
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None



#### Llamadas a la API

Creo las variables de busqueda

In [80]:
api_key = "bd8su7emxovedjjbw00sz1vu8kgluvwc"
api_secret = "yx07uav2PwqX"
country = "es"
operation = "rent"
property_type = "homes"
center = "37.178,-5.851"
distance = "9000"


In [76]:
'''Cada número de esta lista, es el número de página
dentro de las variables de búsqueda de la llamada.
Como necesito sacar mas de 1000 registros pero el máximo por página son 50,
tendré que ir cambiando de página para conseguirlos.'''
numero_pagina = [1,2,3,4,5]

In [81]:
'''Para ello, mediante un bucle for recorro cada número de esa lista
y por cada uno, creo una llamada cuya variable num_page será dicho numero.
En resultados, añdiremos un diccionario por cada llamada.'''
resultados = {}
for numero in numero_pagina:
    nombre_variable = f"result_{numero}"
    resultados[nombre_variable] = search_properties(api_key, api_secret, country, operation, property_type, center, distance,num_page=numero)

In [82]:
'''Mediante un bucle for, vuelvo a recorrer la lista de numeros
para crear de cada diccionario una dataframe y adjudicarle cada número al nombrte'''
dataframes = {}
for numero in numero_pagina:
    nombre_dataframe = f"df_{numero}"
    dataframes[nombre_dataframe] = pd.DataFrame(resultados[f'result_{numero}']['elementList'])

In [84]:
'''concateno todas las dataframes creadas'''
for key, df in dataframes.items():
    df_total = pd.concat([df_total, df], axis=0)

#### Exportación de la dataframe

In [2]:
directorio_actual = os.getcwd()

In [87]:
df_total.to_csv(os.path.join(directorio_actual, '..', 'data','raw','df_total.csv'))